In [ ]:
# BART QVQ Training

# install dependencies
!pip install torch
!pip install sentencepiece
!pip install datasets
!pip install huggingface_hub

# connect to drive
from google.colab import drive
drive.mount('/content/drive')

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch
from torch import no_grad

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to("cuda")

In [ ]:
ignore_sense = False

In [ ]:
tenpredictions = pd.read_csv("/content/drive/MyDrive/T5/bartbasenoskip10epochs.txt", sep="\t", header=None)
twentypredictions = pd.read_csv("/content/drive/MyDrive/T5/bartbasenoskip20epochs.txt", sep="\t", header=None)
thirtypredictions = pd.read_csv("/content/drive/MyDrive/T5/bartbasenoskip30epochs.txt", sep="\t", header=None)
fiftypredictions = pd.read_csv("/content/drive/MyDrive/T5/bartbasenoskip50epochs.txt", sep="\t", header=None)
hundredpredictions = pd.read_csv("/content/drive/MyDrive/T5/bartbasenoskip100epochs.txt", sep="\t", header=None)

predictions = [tenpredictions, twentypredictions, thirtypredictions, fiftypredictions, hundredpredictions]
correct = pd.read_csv("/content/drive/MyDrive/T5/cleaned2_test_binds.txt", sep="\t", header=None)


# compute accuracy

for p in predictions:
  correct_preds = 0
  for i in range(len(p)):
    # ignore verb sense when comparing, so die-01 equals die-02
    if ignore_sense:
      if "-" in p.iloc[i][0]:
        p.iloc[i][0] = p.iloc[i][0].split("-")[0] + " ".join(p.iloc[i][0].split(" ")[1:])
        #print(p.iloc[i][0])
      if "-" in correct.iloc[i][0]:
        correct.iloc[i][0] = correct.iloc[i][0].split("-")[0] + " ".join(correct.iloc[i][0].split(" ")[1:])
    if p.iloc[i][0] == correct.iloc[i][0]:
      correct_preds += 1
  #print(p)
  accuracy = correct_preds / len(p)
  print("Accuracy:", accuracy)


Accuracy: 0.48412698412698413
Accuracy: 0.5238095238095238
Accuracy: 0.5793650793650794
Accuracy: 0.5317460317460317
Accuracy: 0.5238095238095238


In [ ]:
input_ids = tokenizer("Who developed Skype?", return_tensors="pt").input_ids.to("cuda")
labels = tokenizer("develop-02 (*) :ARG0", return_tensors="pt").input_ids.to("cuda")
loss = model(input_ids=input_ids, labels=labels).loss
loss.item()

10.394545555114746

In [ ]:
train_bind = pd.read_csv("/content/drive/MyDrive/T5/cleaned_train_binds.txt", sep="\t", header=None)
train_sent = pd.read_csv("/content/drive/MyDrive/T5/cleaned_qald9_train.txt", sep="\t", header=None)

test_bind = pd.read_csv("/content/drive/MyDrive/T5/cleaned_test_binds.txt", sep="\t", header=None)
test_sent = pd.read_csv("/content/drive/MyDrive/T5/cleaned_qald9_test.txt", sep="\t", header=None)

In [ ]:
# join two dataframes
train_df = pd.concat([train_sent, train_bind], axis=1)
test_df = pd.concat([test_sent, test_bind], axis=1)

train_df.columns = ["sent", "labels"]
test_df.columns = ["sent", "labels"]

# remove rows with "skip" or "SKIP" anywhere in the label
train_df = train_df[~train_df["labels"].str.contains("skip|SKIP")]
test_df = test_df[~test_df["labels"].str.contains("skip|SKIP")]

# load into huggingface dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# tokenize the data
def tokenize_function(text):
    return tokenizer(
        text,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )["input_ids"].squeeze()

train_inputs = torch.stack(list(map(tokenize_function, train_dataset["sent"])))
train_labels = torch.stack(list(map(tokenize_function, train_dataset["labels"])))
test_inputs = torch.stack(list(map(tokenize_function, test_dataset["sent"])))
test_labels = torch.stack(list(map(tokenize_function, test_dataset["labels"])))
train_labels[train_labels == tokenizer.pad_token_id] = -100
test_labels[test_labels == tokenizer.pad_token_id] = -100
train_dataset = TensorDataset(train_inputs, train_labels)
test_dataset = TensorDataset(test_inputs, test_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
len(train_dataset)

329

In [ ]:
# export test_df labels to file
test_df["labels"].to_csv("/content/drive/MyDrive/T5/cleaned2_test_binds.txt", sep="\t", index=False, header=False)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
EVAL_EVERY = 50
EPOCHS = 100
VERBOSE = True

In [ ]:
def eval():
  model.eval()
  total_eval_loss = 0
  with no_grad():
      for batch in tqdm(test_loader, desc="Evaluating"):
          input_ids, labels = [x.to(model.device) for x in batch]
          outputs = model.generate(input_ids=input_ids, labels=labels)

          if VERBOSE:

            output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"Input: {tokenizer.decode(input_ids[0], skip_special_tokens=True)}")
            print(f"Output: {output_text}")

          # Output to File
          for output in outputs:
              output_text = tokenizer.decode(output, skip_special_tokens=True)
              with open(f"/content/drive/MyDrive/T5/bartbasenoskip{epoch+1}epochs.txt", "a") as f:
                  f.write(f"{output_text}\n")

model.train()
for epoch in range(EPOCHS):
  total_loss = 0
  for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
      input_ids, labels = [x.to(model.device) for x in batch]

      optimizer.zero_grad()
      outputs = model(input_ids=input_ids, labels=labels)
      loss = outputs.loss
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
  print(f"Average Loss: {total_loss / len(train_loader)}")
  if (epoch+1) % EVAL_EVERY == 0 and epoch != 0:
    eval()
    model.train()

Epoch 1/100: 100%|██████████| 42/42 [00:12<00:00,  3.27it/s]


Average Loss: 2.133333119608107


Epoch 2/100: 100%|██████████| 42/42 [00:13<00:00,  3.18it/s]


Average Loss: 0.8085041943760145


Epoch 3/100: 100%|██████████| 42/42 [00:13<00:00,  3.10it/s]


...

Input: Who is the governor of Texas?
Output: govern-01 (*) :ARG0


Evaluating:  94%|█████████▍| 15/16 [00:04<00:00,  3.06it/s]

Input: Who is the novelist of the work a song of ice and fire?
Output: person (*) :domain


Evaluating: 100%|██████████| 16/16 [00:04<00:00,  3.35it/s]

Input: Who was called Rodzilla?
Output: call-01 (*) :ARG0
